In [ ]:
!pip install 'qiskit[visualization]'==1.1.0
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install networkx

In [ ]:
%set_env QXToken=<token>

env: QXToken=59dbc3a98a3879c0f87355cfad792aebd7d3ad5ccfa0865713590c7922f9a919699c705cea0c11e4ecbd470b9cf0dae1f26053ccd3e8a627ae64f34b5a50e702


In [ ]:
service = QiskitRuntimeService(
        channel='ibm_quantum',
        token='<token>'
)
backend = service.least_busy(
    operational=True, min_num_qubits=127, simulator=False
)
print(backend)

<IBMBackend('ibm_kyiv')>


In [ ]:
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.synthesis import LieTrotter
# from qiskit.primitives import Sampler
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler

from qiskit_ibm_runtime.options import EstimatorOptions, DynamicalDecouplingOptions

import numpy as np
import matplotlib.pyplot as plt
import json
from typing import List, Set, Tuple
import math
from qiskit_ibm_runtime import EstimatorV2 as Estimator, EstimatorOptions, Batch

In [ ]:
class GroverMaximalClique:
    def __init__(self, V: int, edges: List[Tuple[int, int]]):
        self.V = V
        self.adj_matrix = np.zeros((V, V), dtype=int)
        for v1, v2 in edges:
            self.adj_matrix[v1][v2] = 1
            self.adj_matrix[v2][v1] = 1

        self.qr_state = QuantumRegister(V, 'state')
        self.qr_data = QuantumRegister(V*V, 'data')
        self.qr_ancilla = QuantumRegister(V*V, 'ancilla')
        self.cr = ClassicalRegister(V, 'measure')
        self.qc = QuantumCircuit(self.qr_state, self.qr_data, self.qr_ancilla, self.cr)

    def initialize_adjacency_data(self):
        A_plus_I = self.adj_matrix + np.eye(self.V)
        for i in range(self.V):
            for j in range(self.V):
                if A_plus_I[i][j] == 1:
                    self.qc.x(self.qr_data[i * self.V + j])

    def apply_V_gate(self):
        for j in range(self.V):
            for i in range(self.V):
                data_idx = i * self.V + j
                ancilla_idx = i * self.V + j
                self.qc.ccx(self.qr_state[j], self.qr_data[data_idx], self.qr_ancilla[ancilla_idx])
                self.qc.x(self.qr_state[j])
                self.qc.ccx(self.qr_state[j], self.qr_data[data_idx], self.qr_ancilla[ancilla_idx])
                self.qc.x(self.qr_state[j])
                self.qc.x(self.qr_data[data_idx])
                self.qc.x(self.qr_state[j])
                self.qc.ccx(self.qr_state[j], self.qr_data[data_idx], self.qr_ancilla[ancilla_idx])
                self.qc.x(self.qr_state[j])
                self.qc.x(self.qr_data[data_idx])

    def apply_W_gate(self):
        for i in range(self.V):
            controls = [self.qr_ancilla[i * self.V + j] for j in range(self.V)]
            target = self.qr_state[i]
            self.qc.mcx(controls, target)

    def create_oracle(self):
        self.initialize_adjacency_data()
        self.apply_V_gate()
        self.apply_W_gate()
        self.apply_V_gate()
        self.qc.h(self.qr_state[0])
        self.qc.x(self.qr_state)
        self.qc.mcx(self.qr_state[1:], self.qr_state[0])
        self.qc.x(self.qr_state)
        self.qc.h(self.qr_state[0])

    def apply_diffusion(self):
        self.qc.h(self.qr_state)
        self.qc.x(self.qr_state)
        self.qc.h(self.qr_state[-1])
        self.qc.mcx(self.qr_state[:-1], self.qr_state[-1])
        self.qc.h(self.qr_state[-1])
        self.qc.x(self.qr_state)
        self.qc.h(self.qr_state)

    def create_circuit(self, num_iterations: int):
        print(f"Circuit qubits: {self.qc.num_qubits}")
        print(f"State qubits: {len(self.qr_state)}")
        print(f"Data qubits: {len(self.qr_data)}")
        print(f"Ancilla qubits: {len(self.qr_ancilla)}")
        self.qc.h(self.qr_state)
        for _ in range(num_iterations):
            self.create_oracle()
            self.apply_diffusion()

        self.qc.measure(self.qr_state, self.cr)

        return self.qc

def is_maximal_clique(vertices: List[int], adj_matrix: np.ndarray) -> bool:
    for i in vertices:
        for j in vertices:
            if i != j and adj_matrix[i][j] == 0:
                return False

    V = len(adj_matrix)
    for v in range(V):
        if v not in vertices:
            can_add = True
            for u in vertices:
                if adj_matrix[v][u] == 0:
                    can_add = False
                    break
            if can_add:
                return False
    return True

def find_maximal_cliques(V: int, edges: List[Tuple[int, int]]) -> List[List[int]]:
    adj_matrix = np.zeros((V, V), dtype=int)
    for v1, v2 in edges:
        adj_matrix[v1][v2] = 1
        adj_matrix[v2][v1] = 1

    grover = GroverMaximalClique(V, edges)
    num_iterations = max(1, int(math.pi/4 * math.sqrt(2**V)))
    circuit = grover.create_circuit(num_iterations)
    backend = service.least_busy(
      operational=True, min_num_qubits=circuit.num_qubits, simulator=False
    )
    print(backend)
    '''
    paulis = ["".join("Z" if i == q else "I" for i in range(circuit.num_qubits)) for q in range(circuit.num_qubits)]
    abstract_observables = [SparsePauliOp(pauli) for pauli in paulis]
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    circuit = pm.run(circuit)
    layout = circuit.layout
    # RUN THE CIRCUIT
    target_observables = [abs_obs.apply_layout(layout=layout) for abs_obs in abstract_observables]
    with Batch(backend=backend) as batch:
      pub = (circuit, target_observables)
      default_shots = 10_000
      options = EstimatorOptions()
      # options.optimization_level = 0
      options.resilience_level = 0
      options.default_shots = default_shots
      estimator = Estimator(options=options)
      job = estimator.run(pubs=[pub])
      result = job.result()
      print(result[0])
      counts = result[0].data.evs
    '''
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    circuit = pm.run(circuit)
    with Session(backend=backend) as session:
      sampler = Sampler(mode=session)
      job = sampler.run([circuit])
      result = job.result()[0]
      print(result.data)
      print(f"Sampler job ID: {job.job_id()}")
      print(f"Counts: {result.data.measure.get_counts()}")
      counts = result.data.measure.get_counts()
    '''
    sampler = Sampler()
    job = sampler.run([circuit], shots=1000)
    print(job.result())
    counts = job.result().quasi_dists[0]
    print(counts)
    # counts = {k: round(v * 1000) for k, v in counts.items()}
    counts = {format(k, '03b'): round(v * 1000) for k, v in counts.items()}
    #counts = [int(round(quasi*1000)) for quasi in counts]
    print(counts)
    '''

    cliques = []
    for bitstring in counts:
        if counts[bitstring] > 5:
            vertices = [i for i, bit in enumerate(reversed(bitstring)) if bit == '1']
            if vertices and is_maximal_clique(vertices, adj_matrix):
                cliques.append(sorted(vertices))

    unique_cliques = [list(x) for x in set(tuple(x) for x in cliques)]
    return sorted(unique_cliques)

V, E = map(int, input().strip().split())
edges = []
for _ in range(E):
    v1, v2 = map(int, input().strip().split())
    edges.append((v1, v2))

result = find_maximal_cliques(V, edges)
print(len(result))
for clique in result:
    print(' '.join(map(str, clique)))

6 7
0 1
1 2
0 2
3 4
4 5
3 5
1 4
Circuit qubits: 78
State qubits: 6
Data qubits: 36
Ancilla qubits: 36
<IBMBackend('ibm_kyiv')>
DataBin(measure=BitArray(<shape=(), num_shots=4096, num_bits=6>))
Sampler job ID: cxmsgs3vw7kg008spv20
Counts: {'010110': 69, '011010': 61, '001000': 54, '111111': 62, '001101': 68, '011011': 81, '100001': 80, '110011': 64, '111110': 86, '110110': 57, '111101': 55, '101001': 62, '110010': 58, '101000': 63, '101110': 55, '110000': 77, '001110': 73, '010010': 63, '101101': 65, '000011': 51, '101100': 68, '000001': 54, '000010': 51, '110111': 67, '100101': 59, '001100': 63, '111010': 76, '100110': 67, '000000': 77, '011100': 66, '011001': 65, '100111': 50, '011000': 68, '010000': 70, '111011': 65, '001111': 46, '100000': 82, '101010': 84, '000111': 67, '011101': 62, '010111': 59, '100010': 61, '110101': 50, '000100': 66, '001011': 67, '100100': 84, '010011': 58, '000101': 56, '011110': 63, '101111': 47, '000110': 69, '001010': 66, '001001': 72, '110001': 52, '1101

In [ ]:
6 7
0 1
1 2
0 2
3 4
4 5
3 5
1 4

In [ ]:
class GroverMaximalClique:
    def __init__(self, V: int, edges: List[Tuple[int, int]]):
        self.V = V
        self.adj_matrix = np.zeros((V, V), dtype=int)
        for v1, v2 in edges:
            self.adj_matrix[v1][v2] = 1
            self.adj_matrix[v2][v1] = 1

        self.qr_state = QuantumRegister(V, 'state')
        self.qr_data = QuantumRegister(V*V, 'data')
        self.qr_ancilla = QuantumRegister(V*V, 'ancilla')
        self.cr = ClassicalRegister(V, 'measure')
        self.qc = QuantumCircuit(self.qr_state, self.qr_data, self.qr_ancilla, self.cr)

    def initialize_adjacency_data(self):
        A_plus_I = self.adj_matrix + np.eye(self.V)
        for i in range(self.V):
            for j in range(self.V):
                if A_plus_I[i][j] == 1:
                    self.qc.x(self.qr_data[i * self.V + j])

    def apply_V_gate(self):
        for j in range(self.V):
            for i in range(self.V):
                data_idx = i * self.V + j
                ancilla_idx = i * self.V + j
                self.qc.ccx(self.qr_state[j], self.qr_data[data_idx], self.qr_ancilla[ancilla_idx])
                self.qc.x(self.qr_state[j])
                self.qc.ccx(self.qr_state[j], self.qr_data[data_idx], self.qr_ancilla[ancilla_idx])
                self.qc.x(self.qr_state[j])
                self.qc.x(self.qr_data[data_idx])
                self.qc.x(self.qr_state[j])
                self.qc.ccx(self.qr_state[j], self.qr_data[data_idx], self.qr_ancilla[ancilla_idx])
                self.qc.x(self.qr_state[j])
                self.qc.x(self.qr_data[data_idx])

    def apply_W_gate(self):
        for i in range(self.V):
            controls = [self.qr_ancilla[i * self.V + j] for j in range(self.V)]
            target = self.qr_state[i]
            self.qc.mcx(controls, target)

    def create_oracle(self):
        self.initialize_adjacency_data()
        self.apply_V_gate()
        self.apply_W_gate()
        self.apply_V_gate()
        self.qc.h(self.qr_state[0])
        self.qc.x(self.qr_state)
        self.qc.mcx(self.qr_state[1:], self.qr_state[0])
        self.qc.x(self.qr_state)
        self.qc.h(self.qr_state[0])

    def apply_diffusion(self):
        self.qc.h(self.qr_state)
        self.qc.x(self.qr_state)
        self.qc.h(self.qr_state[-1])
        self.qc.mcx(self.qr_state[:-1], self.qr_state[-1])
        self.qc.h(self.qr_state[-1])
        self.qc.x(self.qr_state)
        self.qc.h(self.qr_state)

    def create_circuit(self, num_iterations: int):
        print(f"Circuit qubits: {self.qc.num_qubits}")
        print(f"State qubits: {len(self.qr_state)}")
        print(f"Data qubits: {len(self.qr_data)}")
        print(f"Ancilla qubits: {len(self.qr_ancilla)}")
        self.qc.h(self.qr_state)
        for _ in range(num_iterations):
            self.create_oracle()
            self.apply_diffusion()

        self.qc.measure(self.qr_state, self.cr)

        return self.qc

def is_maximal_clique(vertices: List[int], adj_matrix: np.ndarray) -> bool:
    for i in vertices:
        for j in vertices:
            if i != j and adj_matrix[i][j] == 0:
                return False

    V = len(adj_matrix)
    for v in range(V):
        if v not in vertices:
            can_add = True
            for u in vertices:
                if adj_matrix[v][u] == 0:
                    can_add = False
                    break
            if can_add:
                return False
    return True

def find_maximal_cliques(V: int, edges: List[Tuple[int, int]]) -> List[List[int]]:
    adj_matrix = np.zeros((V, V), dtype=int)
    for v1, v2 in edges:
        adj_matrix[v1][v2] = 1
        adj_matrix[v2][v1] = 1

    grover = GroverMaximalClique(V, edges)
    num_iterations = max(1, int(math.pi/4 * math.sqrt(2**V)))
    circuit = grover.create_circuit(num_iterations)
    backend = service.least_busy(
      operational=True, min_num_qubits=circuit.num_qubits, simulator=False
    )
    print(backend)
    paulis = ["".join("Z" if i == q else "I" for i in range(circuit.num_qubits)) for q in range(circuit.num_qubits)]
    abstract_observables = [SparsePauliOp(pauli) for pauli in paulis]
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    circuit = pm.run(circuit)
    layout = circuit.layout
    # RUN THE CIRCUIT
    target_observables = [abs_obs.apply_layout(layout=layout) for abs_obs in abstract_observables]
    with Batch(backend=backend) as batch:
      pub = (circuit, target_observables)
      default_shots = 10_000
      options = EstimatorOptions()
      # options.optimization_level = 0
      options.resilience_level = 0
      options.default_shots = default_shots
      estimator = Estimator(options=options)
      job = estimator.run(pubs=[pub])
      result = job.result()
      print(result[0])
      counts = result[0].data.evs
      print(counts)

    cliques = []
    for bitstring in counts:
        if counts[bitstring] > 5:
            vertices = [i for i, bit in enumerate(reversed(bitstring)) if bit == '1']
            if vertices and is_maximal_clique(vertices, adj_matrix):
                cliques.append(sorted(vertices))

    unique_cliques = [list(x) for x in set(tuple(x) for x in cliques)]
    return sorted(unique_cliques)

V, E = map(int, input().strip().split())
edges = []
for _ in range(E):
    v1, v2 = map(int, input().strip().split())
    edges.append((v1, v2))

result = find_maximal_cliques(V, edges)
print(len(result))
for clique in result:
    print(' '.join(map(str, clique)))

6 7
0 1
1 2
0 2
3 4
4 5
3 5
1 4
Circuit qubits: 78
State qubits: 6
Data qubits: 36
Ancilla qubits: 36
<IBMBackend('ibm_kyiv')>
PubResult(data=DataBin(evs=np.ndarray(<shape=(78,), dtype=float64>), stds=np.ndarray(<shape=(78,), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(78,), dtype=float64>), shape=(78,)), metadata={'shots': 10000, 'target_precision': 0.01, 'circuit_metadata': {}, 'num_randomizations': 1})
[ 1.400e-03  5.084e-01 -8.400e-03 -1.920e-02  1.800e-02  1.460e-02
  8.200e-03  1.060e-02  3.400e-03 -5.600e-03  9.800e-03  4.060e-02
  3.900e-02  1.180e-02  1.840e-02  6.000e-04  2.340e-02  1.500e-02
  2.700e-02  1.860e-02  1.980e-02  1.520e-02  1.792e-01  2.760e-02
 -5.600e-03  5.600e-03 -6.800e-03  4.740e-02  4.620e-02  4.580e-02
  1.180e-02 -2.400e-03  1.860e-02  2.260e-02  1.880e-02  3.320e-02
 -3.496e-01  8.200e-03 -4.338e-01 -5.466e-01  3.140e-02 -4.032e-01
 -8.298e-01 -6.400e-03 -8.644e-01 -5.126e-01  6.200e-03  6.000e-03
  9.200e-03 -8.200e-03  1.800e-02  4.24

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices